<img src="../materials/title.png">

# <center><font color='#58ACFA'> AI Workshop - Część II</font></center>
## <center>Wykorzystanie konwolucyjnych sieci neuronowych do klasyfikacji cech twarzy</center>

## <font color='#58ACFA'>Plan pracy:</font>

#### <font color='#0B3861'> >>> Zbudujemy konwolucyjną sieć neuronową</font>
#### <font color='#0B3861'> >>> Przeprowadzimy klasyfikację obrazów ze względu na wybraną cechę</font>
#### <font color='#0B3861'> >>> Podglądniemy, które z elementów obrazu zadecydowały o wyniku klasyfikacji</font>

### <font color='#0B3861'> Do tego celu posłużymy się danymi pochodzącymi ze zbioru CelebA. Zbiór ten posiada ponad 200 tysięcy zdjęć twarzy znanych celebrytów.</font>

<img src="../materials/dataset.png" width="1000">

### <font color='red'>Uwaga: Miejsca w kodzie do uzupełnienia są oznaczone poprzez "____"</font>

## <font color='#0B3861'>Zaczniemy od zaimportowania niezbędnych bibliotek i ustawienia odpowiednich folderów</font>
##### Wykorzystane funkcje zostaną wytłumaczone w dalszej części notatnika

In [ ]:
!pip install --user -r ../requirements.txt # pobieranie bibliotek z pliku requirements.txt

In [ ]:
# operacje związane ze ścieżkami
import os
from glob import glob

 #operacje na macierzach
import numpy as np

# operacje na danych tabelarycznych
import pandas as pd

# operacje na obrazach
from PIL import Image

# wizualizacja postępu wykonania kodu
from tqdm import tqdm

# tworzenie wykresów i wizualizacja danych
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.cm as cm

# importy związane z budową i treningiem modelu
from sklearn.model_selection import train_test_split
from keras.layers import Dense, Input, Dropout
from keras.layers import Conv2D, Flatten, MaxPooling2D
from keras.models import Model
from keras.utils import plot_model
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras import activations

# weryfikacj jakości predykcji
from sklearn.metrics import classification_report, confusion_matrix

# wizualizacja map aktywacji klas (CAM)
from vis.visualization import visualize_cam, overlay
from vis.utils import utils

# importy do przykładu ze zdjęciem z internetu
from urllib.request import urlopen
from resizeimage import resizeimage

%matplotlib inline

#### Zawsze warto ustawić tak zwane ziarno - określa ono punkt początkowy dla algorytmu generacji liczb pseudolosowych. Dzięki temu przy każdorazowym uruchomieniu notatnika uzyskamy te same wyniki. 
Podpowiedź: ziarno ustawiamy podając dowolną liczbę do funkcji <font color=red>np.random.seed()</font>

In [ ]:
____

In [ ]:
# tworzymy foldery na zdjęcia i modele
dataset_dir = '../img_align_celeba'
model_dir = '../models'

os.makedirs(dataset_dir,exist_ok=True)
os.makedirs(model_dir,exist_ok=True)

labels_path = '../list_attr_celeba.txt'
landmarks_path = '../list_landmarks_align_celeba.txt'

## <font color='#0B3861'>Pobranie i rozpakowanie danych</font>
### Uwaga: Dane obrazowe zostały już przygotowane
*   <font color='red'>Linux</font>  
Dane obrazowe:
```bash
!wget https://www.dropbox.com/s/lpmjzzk26nae9bh/img_align_celeba.zip -P ./..
!unzip -q -d ../img_align_celeba ../img_align_celeba.zip
```
Dane liczbowe:

In [ ]:
!wget https://www.dropbox.com/s/qpjuy9isvm19xsv/list_attr_celeba.txt -P ./..

* Windows  
Pobierz pliki ze strony dropbox i wypakuj plik zip wybranym programem do obsługi archiwów.
https://www.dropbox.com/s/lpmjzzk26nae9bh/img_align_celeba.zip  
https://www.dropbox.com/s/qpjuy9isvm19xsv/list_attr_celeba.txt  
https://www.dropbox.com/s/lmc2ywmuk6c29tt/list_landmarks_align_celeba.txt  

## <font color='#0B3861'>Wczytanie i przygotowanie danych</font>
#### Etykiety dotyczące cech twarzy

In [ ]:
attributes = []
with open(labels_path, 'r') as f:
    f.readline()
    attribute_names = ['fn']+f.readline().strip().split(' ')
    for i, line in enumerate(f):
        fields = line.strip().replace('  ', ' ').split(' ')
        img_name = fields[0]
        if int(img_name[:6]) != i + 1:
            raise ValueError('Parse error.')
        attr_vec = np.array([fields[0]]+[int(x) for x in fields[1:]])
        attributes.append(attr_vec)
        
attributes = np.array(attributes)

### Stworzenie tabeli z cechami i etykietami
#### 1 - cecha obecna na obrazie
#### -1 - brak cechy

In [ ]:
df = pd.DataFrame(data=attributes, columns=attribute_names)
df.head()

### Wyświetl listę wszystkich dostępnych cech

In [ ]:
for feature in df.columns[1:]:
    print(feature)

### Wybranie cechy badanej na warsztatach

In [ ]:
examined_feature = 'Bald'

### W celu przyspieszenia obliczeń wykorzystamy jedynie 10 000 losowo wybranych obrazów. Potrzebujemy zatem 10000 losowo wybranych indeksów, ale w taki sposób, żeby wśród wybranych przykładów nasza cecha pojawiła się w mniej więcej połowie przypadków.

#### W tym celu wykorzystamy funkcję <font color='red'>df.groupby()</font> która przyjmuje nazwę wybranej cechy (examined_feature) do pogrupowania rekordów według tej cechy

In [ ]:
df_balanced = ____(____) # pogrupuj rekordy według badanej cechy
n_samples = np.min([10000//2,df_balanced.size().min()])

print(n_samples) # wyświetlenie ilości przykładów zawierających naszą cechę

### Stworzenie tabeli tylko z wybranymi przykładami

#### Wybranie próbki dla każdej z wartości etykiet binarnych

In [ ]:
df_balanced = df_balanced.apply(lambda x: x.sample(n_samples))

#### Ustaw kolumnę 'fn' jako indeks za pomocą <font color='red'>df.set_index()</font> i pozostaw jedynie kolumnę z interesującą nas cechą

In [ ]:
df_balanced = df_balanced.____(____)[examined_feature]

#### Przekształcenie obiektu po grupowaniu z powrotem do tabeli i kolejny restart indeksu w celu uzyskania 'fn' jako osobnej kolumny

In [ ]:
df_balanced = (df_balanced.to_frame()[examined_feature]).reset_index() 

#### Metoda <font color='red'> df.head(n=5)</font> pozwala na wyświetlenie pierwszych 5 wierszy tabeli. Wyświetl 20 pierwszych wierszy. 

In [ ]:
df_balanced.____(____) # wyświetl 20 pierwszych wierszy

#### Całkowita ilość wybranych obrazów:

In [ ]:
print('Mamy',str(len(df_balanced)),'obrazów')

### Przygotowanie etykiet

In [ ]:
labels = (df_balanced[examined_feature].values) == '1'

#### Sprawdź ilość przykładów zawierających naszą cechę oraz ilość przykładów bez tej cechy

In [ ]:
print(np.sum(labels==True),np.sum(labels==False))

### Wczytanie i przygotowanie obrazów

#### Razem z wczytaniem danych  (<font color='red'>Image.open()</font>) przygotujemy je do wykorzystania: przytniemy krawędzie (<font color='red'>crop()</font>) i zmniejszymy ich rozmiar (<font color='red'>resize()</font>), co znacznie przyspieszy trening modelu.
#### Zastosuj wylosowane indeksy w celu wczytania zbioru danych do pracy. Przytnij obraz wprowadzając wartości 9, 13, 169 i 205 (koordynaty pikseli) w odpowednie miejsce. Następnie zmniejsz obraz do rozmiaru 80x96

In [ ]:
images = np.array(
        [np.array(Image.open(os.path.join(dataset_dir,x['fn'])).
                  crop(box=(____)).resize((____)))
         for i,x in tqdm(df_balanced.iterrows())]).astype(np.uint8)

#### Żeby upewnić się, że wszystko poszło dobrze, wyświetl wymiary obrazów 

In [ ]:
____

### Wyświetl przykładowe obrazy osób bez wybranej cechy oraz osób posiadających daną cechę
##### Ustaw wielkość figur na 10x10

In [ ]:
plt.figure(figsize=(____,____))
for i in range(1,10):
    plt.subplot(3,3,i)
    plt.title(examined_feature if labels[i]==1 else '~'+examined_feature)
    plt.axis('off')
    plt.imshow(np.squeeze(images[i]))
plt.show()
plt.figure(figsize=(____,____))
for i in range(1,10):
    plt.subplot(3,3,i)
    plt.title(examined_feature if labels[len(labels) - i]==1 else '~'+examined_feature)
    plt.axis('off')
    plt.imshow(np.squeeze(images[len(labels) - i]))
plt.show()

## <font color='#0B3861'>Budowa modelu konwolucyjnej sieci neuronowej (CNN)</font>

#### Na początku zdefiniujemy parametry określające model CNN i jego trening

In [ ]:
input_shape = images.shape[1:4] #wielkość obrazu wejściowego, wykorzystywana dla budowy grafu SN

""" Parametry określające trening modelu """
batch_size = 128 # liczba obrazów wykorzystywana do jednego kroku treningu SN
optimizer = 'adam' # funkcja odpowiadająca za redukcję błędu sieci
loss_fn = 'sparse_categorical_crossentropy' # funkcja służąca do obliczenia o ile różni się wartość oczekiwana od otrzymanej na wyjściu SN
epochs = 30 # maksymalna liczba epok, czyli ile razy sieć będzie widziała dane treningowe
model_path = os.path.join(model_dir,'celeb_cnn_v2_'+examined_feature+'.h5') # ustawienie ścieżki do pliku z modelem

### Na podstawie poniższego schematu i wskazówek zbuduj model konwolucyjnej sieci neuronowej (uzupełnij brakujące części kodu) 

<img src="../materials/cnn.png" width="1000">

<img src="../materials/cnn_legenda.png" width="400">

### <font color='red'> Ważne wskazówki </font>:
*   Zwróć uwagę na kolory warstw na powyższym schemacie i nazwy w nawiasach
*   Brakujące warstwy uzupełniaj analogicznie do wpisanych już przykładów
*   Uzupełnij brakujące pola "____" w wyznaczonych miejscach
*   Funkcja aktywacji to <font color='red'>'relu'</font> (za wyjątkiem ostatniej, gdzie jest 'softmax')
*   Dla każdej kolejnej warstwy konwolucyjnej liczbę filtrów <font color='red'>zwiększ dwukrotnie w stosunku do poprzedniej</font> 
*   Dla każdej warstwy konwolucyjnej ustaw wielkość filtru <font color='red'>3x3</font>
*   Dla każdej warstwy głosującej ustaw wielkość filtru <font color='red'>2x2</font>
*   Dla każdej warstwy Dropout (z wyjątkiem ostatniej) <font color='red'>25%</font> neuronów zostaje wyłączona 

In [ ]:
inputs = Input(shape=input_shape, name='input')

x = Conv2D(64, (3, 3), activation='relu')(inputs)
x = MaxPooling2D(pool_size=(2, 2))(x)
x = Dropout(0.25)(x)

x = ____(____)(x)
x = ____(____)(x)
x = ____(____)(x)

x = ____(____)(x)
x = ____(____)(x)
x = ____(____)(x)

x = Flatten()(x)

x = ____(256, activation=____)(x)
x = Dropout(0.5)(x)
x = ____(2, activation='softmax')(x)

model = Model(inputs=inputs, outputs=x, name='CNN')
model.compile(optimizer=optimizer, loss=loss_fn, metrics=['acc'])

## <font color='#0B3861'>Trening konwolucyjnej sieci neuronowej</font>

### Podział danych na testowe i treningowe
#### Podział danych jest realizowany za pomocą funkcji <font color='red'>train_test_split</font>, która przyjmuje dane do podziału, ich etykiety oraz procent przykładów które będą przydzielone do zbioru testowego <font color='red'>test_size</font>

In [ ]:
X_train, X_test, y_train, y_test = ____(images.astype('float32')/255, labels, ____=0.2) # uzupełnij kod do podziału danych

#### Powyższa funkcja wyprodukowała 4 zmienne: X_train i X_test zawierające odpowiednio dane treningowe i testowe oraz y_train, y_test będące etykietami dla każdego z przykładów w zbiorach danych

### Standaryzacja danych
#### Dokończ standaryzowanie obrazów, w każdym z kanałów RGB (3), wykorzystując funkcje biblioteki numpy wyliczające średnią i odchylenie standardowe (<font color='red'>np.mean(), np.std()</font>) oraz poniższy wzór:
\begin{equation}
X = \frac{X - mean(X)}{std(X)}
\end{equation}

In [ ]:
X_mean = []
X_std = []
for i in range(3):
    # zapisanie średnich i odchyleń z danych - będą potrzebne później
    X_mean.append(np.mean(X_train[:,:,:,i]))
    X_std.append(np.std(X_train[:,:,:,i]))
    
    # uzupełnij poniższe wzory standaryzujące dane treningowe i testowe
    X_train[:,:,:,i] = (X_train[:,:,:,i] - ____(____))/____(____)
    X_test[:,:,:,i] = (X_test[:,:,:,i] - ____(____))/____(____)

#### Wprowadzenie parametrów treningowych i funkcji monitorujących przebieg treningu

In [ ]:
early_stop = EarlyStopping(restore_best_weights=True, patience=3) # zatrzyma uczenie jeśli nie będzie poprawy przez 3 epoki
checkpt = ModelCheckpoint(model_path, save_best_only=True, save_weights_only=True) # zapisze wagi dla najlepszej epoki

### Trening CNN
#### Dopisz w odpowiednich miejscach brakujące dane i etykiety (dane testowe jako walidacyjne)

In [ ]:
history = model.fit(X_train, ____,
                  epochs=epochs,
                  batch_size=batch_size,
                  validation_data=(____, y_test),
                  verbose=2,
                  callbacks = [early_stop, checkpt])

## <font color='#0B3861'>Weryfikacja rezultatów</font>
#### Wczytanie zapisanych wag z epoki, dla której wartość funkcji straty była najniższa

In [ ]:
model.load_weights(model_path)

#### Weryfikacja jakości predykcji

In [ ]:
y_pred = np.argmax(model.predict(X_test),axis=-1)

In [ ]:
print(confusion_matrix(y_test, y_pred)) # macierz błędów klasyfikacji

In [ ]:
print(classification_report(____, ____)) # uzupełnij raport

## <font color='#0B3861'>Wizualizacja CAM</font>

### CAMy to mapy cieplne wskazujące obszary obrazu, które przyczyniły się najbardziej do wyniku klasyfikacji

In [ ]:
# wymagane modyfikacje
model_vis = model
model_vis.layers[-1].activation = activations.linear
model_vis = utils.apply_modifications(model_vis)

In [ ]:
%matplotlib inline
plt.figure(figsize=(10,10))
for i in range(1,10):
    plt.subplot(3,3,i)
    plt.title(examined_feature if labels[i]==1 else '~'+examined_feature)
    grads = visualize_cam(model_vis, -1, filter_indices=int(labels[i]), 
                          seed_input=images[i], backprop_modifier='guided')    
    plt.imshow(overlay(cm.jet(grads)[:,:,:3], images[i]/255))
plt.show()

plt.figure(figsize=(10,10))
for i in range(1,10):
    plt.subplot(3,3,i)
    plt.title(examined_feature if labels[len(labels) - i]==1 else '~'+examined_feature)
    grads = visualize_cam(model_vis, -1, filter_indices=int(labels[[len(labels) - i]]), 
                          seed_input=images[len(labels) - i], backprop_modifier='guided')    
    plt.imshow(overlay(cm.jet(grads)[:,:,:3], images[len(labels) - i]/255))
plt.show()

### <font color='#58ACFA'>>>> Co można wywnioskować na podstawie uzyskanych wizualizacji CAMów?</font>

## <font color='#0B3861'>Sprawdzenie działania na zdjęciu spoza zbioru danych - przykład z internetu</font>

In [ ]:
img = Image.open(urlopen('http://www.afternoondc.in/Thumbnails.aspx?Filename=2018319214726.jpg'))
img

#### Wyświetl rozmiar obrazu

#### Za pomocą funkcji <font color='red'>resizeimage.resize_cover</font> zmniejsz obraz do rozmiaru <font color='red'>80x96</font>

In [ ]:
img = ____(img, [____, ____])
img

### Przetworzenie obrazu do formy akceptowanej przez sieć neuronową

In [ ]:
img_float = np.array(img).astype(np.float32)
img_std = img_float.copy()
for i in range(3):
    img_std[:,:,i] = (img_std[:,:,i]-X_mean[i])/X_std[i]
img_std = np.expand_dims(img_std,0)

img_std.shape # wyświetlenie końcowych wymiarów obrazu (po przetworzeniu)

### Predykcja sieci neuronowej dla przygotowanego obrazu za pomocą <font color='red'>predict()</font>

In [ ]:
img_pred = model.____(____)

### Wizualizacja CAM dla obrazu

In [ ]:
grads = visualize_cam(model_vis, -1, filter_indices=1, 
                      seed_input=np.squeeze(img_std), backprop_modifier='guided')    
plt.imshow(overlay(cm.jet(grads)[:,:,:3], img_float/255))
plt.show()

## <font color='#0B3861'>Twój przykład. Znajdź obraz na internecie i sprawdź odpowiedź sieci neuronowej :)</font>

In [ ]:
from urllib.request import urlopen
from resizeimage import resizeimage

img = Image.open(urlopen(' tutaj link do obrazka '))
img

In [ ]:
img = ____(img, [____, ____]) # zmniejszenie rozmiaru resizeimage.resize_cover
img

In [ ]:
img_float = np.array(img).astype(np.float32)
img_std = img_float.copy()
for i in range(3):
    img_std[:,:,i] = (img_std[:,:,i]-X_mean[i])/X_std[i]
img_std = np.expand_dims(img_std,0)
img_std.shape

In [ ]:
img_pred = model.____(____) # predykcja

In [ ]:
grads = visualize_cam(model_vis, -1, filter_indices=1, 
                      seed_input=np.squeeze(img_std), backprop_modifier='guided')    
plt.imshow(overlay(cm.jet(grads)[:,:,:3], img_float/255))
plt.show()